# 动态图

从飞桨开源框架2.0beta版本开始，飞桨默认为用户开启了动态图模式。在这种模式下，每次执行一个运算，可以立即得到结果（而不是事先定义好网络结构，然后再执行）。

在动态图模式下，您可以更加方便的组织代码，更容易的调试程序，本示例教程将向你介绍飞桨的动态图的使用。


# 设置环境

我们将使用飞桨2.0beta版本，并确认已经开启了动态图模式。

In [6]:
import paddle
import paddle.nn.functional as F
import numpy as np

paddle.disable_static()
print(paddle.__version__)
print(paddle.__git_commit__)


0.0.0
edf5f3173a25ae2230e9619ab5426317b4bd7cde


# 基本用法

在动态图模式下，您可以直接运行一个飞桨提供的API，它会立刻返回结果到python。不再需要首先创建一个计算图，然后再给定数据去运行。

In [2]:
a = paddle.randn([4, 2])
b = paddle.arange(1, 3, dtype='float32')

print(a.numpy())
print(b.numpy())

c = a + b
print(c.numpy())

d = paddle.matmul(a, b)
print(d.numpy())

[[-1.4396907  -0.59741247]
 [ 1.4717171  -0.06998838]
 [-1.2790705   0.4278928 ]
 [ 1.1862146  -1.895377  ]]
[1. 2.]
[[-0.4396907   1.4025875 ]
 [ 2.4717171   1.9300116 ]
 [-0.2790705   2.4278927 ]
 [ 2.1862144   0.10462296]]
[-2.6345158  1.3317404 -0.4232849 -2.6045394]


# 使用python的控制流

动态图模式下，您可以使用python的条件判断和循环，这类控制语句来执行神经网络的计算。（不再需要`cond`, `loop`这类OP)


In [3]:
a = paddle.to_tensor(np.array([1, 2, 3]))
b = paddle.to_tensor(np.array([4, 5, 6]))

for i in range(10):
    r = paddle.rand([1,])
    if r > 0.5:
        c = paddle.pow(a, i) + b
        print("{} +> {}".format(i, c.numpy()))
    else:
        c = paddle.pow(a, i) - b
        print("{} -> {}".format(i, c.numpy()))


0 +> [5 6 7]
1 -> [-3 -3 -3]
2 -> [-3 -1  3]
3 +> [ 5 13 33]
4 -> [-3 11 75]
5 +> [  5  37 249]
6 -> [ -3  59 723]
7 -> [  -3  123 2181]
8 -> [  -3  251 6555]
9 -> [   -3   507 19677]


# 构建更加灵活的网络

- 使用动态图可以用来创建更加灵活的网络，比如根据控制流选择不同的分支网络，和方便的构建权重共享的网络。接下来我们来看一个具体的例子，在这个例子中，第二个线性变换只有0.5的可能性会运行。



In [4]:
class MyModel(paddle.nn.Layer):
    def __init__(self, input_size, hidden_size):
        super(MyModel, self).__init__()
        self.linear1 = paddle.nn.Linear(input_size, hidden_size)
        self.linear2 = paddle.nn.Linear(hidden_size, hidden_size)
        self.linear3 = paddle.nn.Linear(hidden_size, 1)

    def forward(self, inputs):
        x = self.linear1(inputs)
        x = F.relu(x)

        if paddle.rand([1,]) > 0.5: 
            x = self.linear2(x)
            x = F.relu(x)

        x = self.linear3(x)
        
        return x     

In [5]:
total_data, batch_size, input_size, hidden_size = 1000, 64, 128, 256

x_data = np.random.randn(total_data, input_size).astype(np.float32)
y_data = np.random.randn(total_data, 1).astype(np.float32)

model = MyModel(input_size, hidden_size)

loss_fn = paddle.nn.MSELoss(reduction='sum')
optimizer = paddle.optimizer.SGD(learning_rate=0.0001, 
                                 parameter_list=model.parameters())

for t in range(200 * (total_data // batch_size)):
    idx = np.random.choice(total_data, batch_size, replace=False)
    x = paddle.to_tensor(x_data[idx,:])
    y = paddle.to_tensor(y_data[idx,:])
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t % 200 == 0:
        print(t, loss.numpy())

    loss.backward()
    optimizer.minimize(loss)
    model.clear_gradients()

0 [87.28088]
200 [57.775795]
400 [42.70884]
600 [45.509155]
800 [29.966158]
1000 [11.885025]
1200 [16.888378]
1400 [3.5780585]
1600 [5.3149533]
1800 [4.501356]
2000 [3.022315]
2200 [1.7214009]
2400 [0.3694626]
2600 [0.31249344]
2800 [0.1450614]


# The end

可以看到使用动态图带来了更灵活易用的方式来组网和训练。